In [19]:
import warnings
warnings.simplefilter("ignore")

import pandas as pd
import numpy as np

import catboost as cb

from sklearn.metrics import f1_score, roc_auc_score
from sklearn.model_selection import train_test_split

import datetime
from datetime import datetime, date

In [2]:
data = pd.read_csv('assignment_2_train.csv')

In [3]:
data.head(2)

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
0,2987000,0,86400,68.5,W,13926,NaN,150.0,discover,142.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2987001,0,86401,29.0,W,2755,404.0,150.0,mastercard,102.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


**Задание 0:** выбрать любую модель машнного обучения и зафиксировать любой тип валидации. Обучить базовую модель и зафиксировать базовое качество модели. В каждом следующем задании нужно будет обучить выбранную модель и оценивать ее качество на зафиксированной схеме валидации. После каждого задания, требуется сделать вывод о достигаемом качестве модели, по сравнению с качестом из предыдущего шага.

In [8]:
target = data['isFraud']

data.drop(['isFraud', 'TransactionID'], axis=1, inplace=True)

In [9]:
categorical_features = data.select_dtypes(include=[np.object])
print(f'Всего категориальных признаков {categorical_features.shape[1]}')
# categorical_features = categorical_features.columns.tolist()

Всего категориальных признаков 14


In [10]:
for col in categorical_features:
    data[col] = data[col].astype(str)

In [11]:
x_train, x_valid, y_train, y_valid = train_test_split(
    data, target, train_size=0.7, shuffle=True, random_state=1,
)

In [13]:
train_pool = cb.Pool(x_train, y_train, cat_features=categorical_features)
valid_pool = cb.Pool(x_valid, y_valid, cat_features=categorical_features)

In [34]:
cb_params = {
#     "n_estimators": 3000,
    "loss_function": "Logloss",
    "eval_metric": "AUC",
    "task_type": "CPU",
    "max_bin": 20,
    "verbose": 100,
    "max_depth": 6,
    "l2_leaf_reg": 100,
    "early_stopping_rounds": 100,
    "thread_count": 6,
    "random_seed": 42,
    "cat_features":categorical_features
}
model = cb.CatBoostClassifier(**cb_params)
model.fit(x_train, y_train, eval_set=[(x_train, y_train), (x_valid, y_valid)])

0:	test: 0.6645044	test1: 0.6640967	best: 0.6640967 (0)	total: 302ms	remaining: 5m 1s
100:	test: 0.8626380	test1: 0.8568925	best: 0.8568925 (100)	total: 26.1s	remaining: 3m 52s
200:	test: 0.8851982	test1: 0.8778457	best: 0.8778457 (200)	total: 52s	remaining: 3m 26s
300:	test: 0.8968462	test1: 0.8875002	best: 0.8875002 (300)	total: 1m 18s	remaining: 3m 1s
400:	test: 0.9003293	test1: 0.8903009	best: 0.8903009 (400)	total: 1m 44s	remaining: 2m 35s
500:	test: 0.9030912	test1: 0.8922876	best: 0.8922876 (500)	total: 2m 9s	remaining: 2m 8s
600:	test: 0.9062248	test1: 0.8946995	best: 0.8947030 (599)	total: 2m 34s	remaining: 1m 42s
700:	test: 0.9078557	test1: 0.8959960	best: 0.8960067 (666)	total: 3m	remaining: 1m 16s
800:	test: 0.9079064	test1: 0.8960219	best: 0.8960221 (797)	total: 3m 26s	remaining: 51.2s
900:	test: 0.9081127	test1: 0.8962543	best: 0.8962553 (854)	total: 3m 51s	remaining: 25.4s
999:	test: 0.9086295	test1: 0.8967270	best: 0.8967292 (965)	total: 4m 16s	remaining: 0us

bestTest 

In [17]:
cb_params = {
    "n_estimators": 500,
    "loss_function": "Logloss",
    "eval_metric": "AUC",
    "task_type": "CPU",
    "max_bin": 20,
    "verbose": 100,
    "max_depth": 6,
    "l2_leaf_reg": 100,
    "early_stopping_rounds": 100,
    "thread_count": 6,
    "random_seed": 42,
    "cat_features":categorical_features
}
model = cb.CatBoostClassifier(**cb_params)
model.fit(x_train, y_train, eval_set=[(x_train, y_train), (x_valid, y_valid)])

0:	test: 0.6645044	test1: 0.6640967	best: 0.6640967 (0)	total: 287ms	remaining: 2m 22s
100:	test: 0.8626380	test1: 0.8568925	best: 0.8568925 (100)	total: 26.6s	remaining: 1m 45s
200:	test: 0.8851982	test1: 0.8778457	best: 0.8778457 (200)	total: 53.2s	remaining: 1m 19s
300:	test: 0.8968462	test1: 0.8875002	best: 0.8875002 (300)	total: 1m 20s	remaining: 53.1s
400:	test: 0.9003293	test1: 0.8903009	best: 0.8903009 (400)	total: 1m 46s	remaining: 26.3s
499:	test: 0.9029770	test1: 0.8921661	best: 0.8921661 (499)	total: 2m 11s	remaining: 0us

bestTest = 0.8921660995
bestIteration = 499



**Задание 1:** признак TransactionDT - это смещение в секундах относительно базовой даты. Базовая дата - 2017-12-01, преобразовать признак TransactionDT в datetime, прибавив к базовой дате исходное значение признака. Из полученного признака выделить год, месяц, день недели, час, день.

In [63]:
base_date = int(datetime(2017, 12, 1).timestamp())
print(base_date)

1512075600


In [72]:
def change_date(df, feature):
    df[feature] = pd.to_datetime(df[feature] + base_date)
    df['Year'] = df[feature].dt.year
    df['Month'] = df[feature].dt.month
    df['Day_of_week'] = df[feature].dt.weekday
    df['Hour'] = df[feature].dt.hour
    df['Day'] = df[feature].dt.day
    df.drop([feature], axis=1, inplace=True)
    return df

In [73]:
x_train = change_date(x_train, "TransactionDT")
x_valid = change_date(x_valid, "TransactionDT")

In [74]:
train_pool = cb.Pool(x_train, y_train, cat_features=categorical_features)
valid_pool = cb.Pool(x_valid, y_valid, cat_features=categorical_features)

In [75]:
cb_params = {
#     "n_estimators": 3000,
    "loss_function": "Logloss",
    "eval_metric": "AUC",
    "task_type": "CPU",
    "max_bin": 20,
    "verbose": 100,
    "max_depth": 6,
    "l2_leaf_reg": 100,
    "early_stopping_rounds": 100,
    "thread_count": 6,
    "random_seed": 42,
    "cat_features":categorical_features
}
model = cb.CatBoostClassifier(**cb_params)
model.fit(x_train, y_train, eval_set=[(x_train, y_train), (x_valid, y_valid)])

0:	test: 0.6229201	test1: 0.6242434	best: 0.6242434 (0)	total: 298ms	remaining: 4m 57s
100:	test: 0.8641673	test1: 0.8583656	best: 0.8583656 (100)	total: 26.7s	remaining: 3m 57s
200:	test: 0.8781339	test1: 0.8717821	best: 0.8717821 (200)	total: 52.6s	remaining: 3m 28s
300:	test: 0.8902962	test1: 0.8816370	best: 0.8816370 (300)	total: 1m 19s	remaining: 3m 4s
400:	test: 0.8938344	test1: 0.8845752	best: 0.8845752 (400)	total: 1m 45s	remaining: 2m 37s
500:	test: 0.8961948	test1: 0.8863814	best: 0.8863872 (497)	total: 2m 10s	remaining: 2m 10s
600:	test: 0.8985396	test1: 0.8877131	best: 0.8877131 (600)	total: 2m 36s	remaining: 1m 43s
700:	test: 0.9007850	test1: 0.8891580	best: 0.8891580 (700)	total: 3m 2s	remaining: 1m 17s
800:	test: 0.9008832	test1: 0.8892245	best: 0.8892254 (791)	total: 3m 28s	remaining: 51.7s
900:	test: 0.9009137	test1: 0.8892476	best: 0.8892492 (890)	total: 3m 52s	remaining: 25.6s
999:	test: 0.9009508	test1: 0.8892921	best: 0.8892922 (994)	total: 4m 16s	remaining: 0us

b